In [1]:
# pip install keras-self-attention
import os, time, gc, sys, io
import datetime, pickle
import warnings, random, pdb

import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import mpl_toolkits.axisartist as AA
import os
import pickle

from scipy import stats
from pandas import Series
from numpy.lib.stride_tricks import sliding_window_view

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

### General functions

In [21]:
def windowing(dfX,win,nvar,idx):
    lX = np.lib.stride_tricks.sliding_window_view(dfX.to_numpy(), window_shape = win)[::nvar]
    mX = pd.DataFrame.from_records(lX)
    mX.set_index(idx,drop=True, inplace=True)
    return(mX)

### Load Data

In [4]:
path = "Datasets/"
stock_list= ["AAPL", "ADBE", "AMZN", "AVGO", "CMCSA", "COST", "CSCO", "GOOG", "GOOGL", "META", "MSFT", "NVDA", "PEP", "TMUS", "TSLA"]

df_dict = {}
df_result = pd.DataFrame(columns = ['MSEP', 'MSEY', 'Stock','Algo'])
tot_res = {}
#
for stock in stock_list:
    fich = path + stock +' US Equity_060124.csv'
    df_dict[stock] = pd.read_csv(fich, sep = ",",index_col=0,parse_dates=True)
    df_dict[stock]['PX_TREND'] = 2*(df_dict[stock]['PX_LAST'] - \
                                    df_dict[stock]['PX_OPEN'])/(df_dict[stock]['PX_OPEN'] + \
                                                                df_dict[stock]['PX_LAST'])
    df_dict[stock]['PX_VTREND']= df_dict[stock]['PX_TREND'] * df_dict[stock]['VOLUME']
    nans = sum(df_dict[stock]['PX_OPEN'].isna())*100/df_dict[stock].shape[0]
    df_dict[stock] = df_dict[stock].dropna()
    # Adopted policy is to drop the NANs as they represent holidays or working off days
    print('{}: NaNs are about {}% in the original dataset. Size: {}'.format(\
                                    stock,str(round(nans)),str(df_dict[stock].shape)))
    tot_res['INP'] = df_dict

df = df_dict['TSLA']
df.head(1)

AAPL: NaNs are about 4% in the original dataset. Size: (2257, 18)
ADBE: NaNs are about 4% in the original dataset. Size: (1790, 18)
AMZN: NaNs are about 4% in the original dataset. Size: (2239, 18)
AVGO: NaNs are about 4% in the original dataset. Size: (1613, 18)
CMCSA: NaNs are about 4% in the original dataset. Size: (1901, 18)
COST: NaNs are about 4% in the original dataset. Size: (1388, 18)
CSCO: NaNs are about 4% in the original dataset. Size: (1994, 18)
GOOG: NaNs are about 4% in the original dataset. Size: (2251, 18)
GOOGL: NaNs are about 4% in the original dataset. Size: (2251, 18)
META: NaNs are about 4% in the original dataset. Size: (2251, 18)
MSFT: NaNs are about 4% in the original dataset. Size: (2253, 18)
NVDA: NaNs are about 4% in the original dataset. Size: (2088, 18)
PEP: NaNs are about 4% in the original dataset. Size: (1836, 18)
TMUS: NaNs are about 4% in the original dataset. Size: (1774, 18)
TSLA: NaNs are about 4% in the original dataset. Size: (2246, 18)


,TWITTER_SENTIMENT_DAILY_AVG,TWITTER_PUBLICATION_COUNT,TWITTER_NEG_SENTIMENT_COUNT,TWITTER_POS_SENTIMENT_COUNT,TWITTER_NEUTRAL_SENTIMENT_CNT,NEWS_SENTIMENT_DAILY_AVG,NEWS_PUBLICATION_COUNT,NEWS_NEG_SENTIMENT_COUNT,NEWS_POS_SENTIMENT_COUNT,NEWS_NEUTRAL_SENTIMENT_COUNT,PX_HIGH,PX_LOW,PX_OPEN,PX_LAST,VOLUME,RSI_14D,PX_TREND,PX_VTREND
date,,,,,,,,,,,,,,,,,,
2015-01-02,0.0018,222.0,6.0,9.0,192.0,0.0569,139.0,3.0,7.0,41.0,14.8833,14.2173,14.858,14.6207,71466645.0,59.8008,-0.0161,-1.150596e+06


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2246 entries, 2015-01-02 to 2023-12-29
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   TWITTER_SENTIMENT_DAILY_AVG    2246 non-null   float64
 1   TWITTER_PUBLICATION_COUNT      2246 non-null   float64
 2   TWITTER_NEG_SENTIMENT_COUNT    2246 non-null   float64
 3   TWITTER_POS_SENTIMENT_COUNT    2246 non-null   float64
 4   TWITTER_NEUTRAL_SENTIMENT_CNT  2246 non-null   float64
 5   NEWS_SENTIMENT_DAILY_AVG       2246 non-null   float64
 6   NEWS_PUBLICATION_COUNT         2246 non-null   float64
 7   NEWS_NEG_SENTIMENT_COUNT       2246 non-null   float64
 8   NEWS_POS_SENTIMENT_COUNT       2246 non-null   float64
 9   NEWS_NEUTRAL_SENTIMENT_COUNT   2246 non-null   float64
 10  PX_HIGH                        2246 non-null   float64
 11  PX_LOW                         2246 non-null   float64
 12  PX_OPEN                       

In [20]:
df.describe()

,TWITTER_SENTIMENT_DAILY_AVG,TWITTER_PUBLICATION_COUNT,TWITTER_NEG_SENTIMENT_COUNT,TWITTER_POS_SENTIMENT_COUNT,TWITTER_NEUTRAL_SENTIMENT_CNT,NEWS_SENTIMENT_DAILY_AVG,NEWS_PUBLICATION_COUNT,NEWS_NEG_SENTIMENT_COUNT,NEWS_POS_SENTIMENT_COUNT,NEWS_NEUTRAL_SENTIMENT_COUNT,PX_HIGH,PX_LOW,PX_OPEN,PX_LAST,VOLUME,RSI_14D,PX_TREND,PX_VTREND
count,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2246.000000,2.246000e+03,2246.000000,2246.000000,2.246000e+03
mean,-0.011724,2225.603295,243.328139,176.725289,1728.593945,-0.007657,786.969279,25.896260,18.248442,142.836598,105.859928,101.048998,103.517803,103.488381,1.146288e+08,52.864388,0.000178,1.118767e+05
std,0.065884,1892.479825,361.788078,273.836894,1438.994872,0.140265,617.672035,46.322212,32.388767,99.232105,113.747127,108.442276,111.190586,111.090898,7.719474e+07,13.394770,0.028507,6.376670e+06
min,-0.823100,130.000000,1.000000,4.000000,87.000000,-0.888700,67.000000,0.000000,0.000000,0.000000,10.331300,9.403300,9.488000,9.578000,1.065416e+07,16.564100,-0.136635,-8.257751e+07
25%,-0.019600,1147.000000,84.000000,64.000000,928.500000,-0.023575,363.250000,4.000000,3.000000,75.000000,16.958000,16.496500,16.717825,16.712175,6.643042e+07,43.676050,-0.015319,-1.407506e+06
50%,-0.007000,1692.500000,145.000000,108.000000,1346.500000,-0.001300,628.000000,11.000000,8.000000,122.000000,23.577650,22.859700,23.231300,23.210000,9.388688e+07,51.657850,0.000435,2.866695e+04
75%,0.007300,2616.000000,254.000000,192.000000,2024.500000,0.011825,997.250000,27.000000,20.000000,188.000000,219.074525,207.633350,214.079175,214.717500,1.347306e+08,61.836825,0.015759,1.430274e+06
max,0.761400,20994.000000,7424.000000,6005.000000,19651.000000,0.879500,6123.000000,718.000000,351.000000,1189.000000,414.496700,405.666700,411.470000,409.970000,9.140814e+08,94.198000,0.146262,1.036267e+08


### Dates of the data under study

In [9]:
df_dict["TSLA"].index

DatetimeIndex(['2015-01-02', '2015-01-05', '2015-01-06', '2015-01-07',
               '2015-01-08', '2015-01-09', '2015-01-12', '2015-01-13',
               '2015-01-14', '2015-01-15',
               ...
               '2023-12-15', '2023-12-18', '2023-12-19', '2023-12-20',
               '2023-12-21', '2023-12-22', '2023-12-26', '2023-12-27',
               '2023-12-28', '2023-12-29'],
              dtype='datetime64[ns]', name='date', length=2246, freq=None)

### Preprocessing

In [10]:
mpl.rcParams['figure.figsize'] = (18, 10)
mpl.rcParams['axes.grid'] = False
sns.set_style("whitegrid")

print(f"Pandas Version: {pd.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"System Version: {sys.version}")

Pandas Version: 1.5.0
Numpy Version: 1.24.3
System Version: 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)]


In [11]:
win         = 15  # window size
lahead      = [1, 7, 14, 30, 90] # Number of days ahead for prediction.
deep        = 1   # Number of features considered
n_ftrs      = 1   # 1 feature to be predicted
tr_tst      = 0.8 # % of samples used for training
serial_dict = {}

In [12]:
for stock in stock_list:
    serial_dict[stock] = {}
    for ahead in lahead:
        df = df_dict[stock].copy()

        # Window of relevant data
        win_x = np.lib.stride_tricks.sliding_window_view(df.PX_OPEN.to_numpy(), window_shape = win)[::1]

        # Shift the Y axis
        Y= df.iloc[ahead+win:]['PX_OPEN']
        X = pd.DataFrame.from_records(win_x)

        # Cutting out records with no data in Y+ahead respecting the win+ahead offset
        X = X.iloc[:-(ahead+1),:] # Skipping the gap in forecasting values
        X.set_index(df.index[(win-1):-(ahead+1)],drop=True, inplace=True)

        xm = X.mean(axis=1)
        cX = X.sub(X.mean(axis=1), axis=0)

        #  Scaling and storing ranges in vdd
        mnx= round(min(cX.min())) # whole min
        mxx= round(max(cX.max())) # whole max
        vdd= pd.DataFrame({'mean':xm,'min':mnx,'max':mxx}) # DF with ranges
        vdd.set_index(X.index,drop=True,inplace=True)

        # Normalizing X and Y
        cXn= cX.apply(lambda x: (x-mnx)/(mxx-mnx), axis=1)
        cYn= pd.Series([((i-j)-mnx)/(mxx-mnx) for i,j in zip(Y.tolist(),xm.tolist())], index=Y.index)
        cXn = cXn.astype('float32')
        cYn = cYn.astype('float32')

        # Data set preparation for modeling, starting from cXn and cYn
        pmod   = int(cXn.shape[0]*tr_tst)  # 20% reserved for test data
        trainX = cXn.iloc[:pmod,:]
        trainY = cYn.iloc[:pmod]
        testX  = cXn.iloc[pmod:,:]
        testY  = cYn.iloc[pmod:]

        serial_dict[stock][ahead] = {'x':X,'y':Y, 'nx':cXn,'ny':cYn, 'numt':pmod, 'vdd':vdd, \
                                     'trX':trainX,'trY':trainY, 'tsX':testX,'tsY':testY}

#Checking info       
for ahead in lahead:
    print("Prediction {} day(s) ahead.".format(ahead))
    X = serial_dict['AAPL'][ahead]['trX']
    Y = serial_dict['AAPL'][ahead]['trY']
    print("  Var Indep. {}. Rows {}. Var dep samples {}.".format(X.shape[1],X.shape[0],Y.shape[0]))
tot_res['INP_SERIAL'] = serial_dict

Prediction 1 day(s) ahead.
  Var Indep. 15. Rows 1792. Var dep samples 1792.
Prediction 7 day(s) ahead.
  Var Indep. 15. Rows 1788. Var dep samples 1788.
Prediction 14 day(s) ahead.
  Var Indep. 15. Rows 1782. Var dep samples 1782.
Prediction 30 day(s) ahead.
  Var Indep. 15. Rows 1769. Var dep samples 1769.
Prediction 90 day(s) ahead.
  Var Indep. 15. Rows 1721. Var dep samples 1721.


In [61]:
fdat = "/data_processed/{:02}/input-output.pkl".format(win)
lpar  = [win, deep, n_ftrs,tr_tst]

with open(fdat, 'wb') as file:
    pickle.dump(path, file)
    pickle.dump(fdat,file)
    pickle.dump(lahead,file)
    pickle.dump(lpar,file)
    pickle.dump(stock_list,file)
    pickle.dump(tot_res, file)
file.close()

## Multivariate preprocessing for Transformer multi-heading
### Sentiment Analysis, RSI Technical Indicator, News and Price and Volume Trend

#### Data Preparation

In [17]:
print(df_dict.keys()) # Stocks tickers

dict_keys(['AAPL', 'ADBE', 'AMZN', 'AVGO', 'CMCSA', 'COST', 'CSCO', 'GOOG', 'GOOGL', 'META', 'MSFT', 'NVDA', 'PEP', 'TMUS', 'TSLA'])


In [22]:
mwin  = 15  # multivariate window size
mdeep = 8   # multivariate number of features considered
m_ftrs= 1   # multivariate 1 feature to be predicted
mserial_dict = {}

for stock in stock_list:
    mserial_dict[stock] = {}
    df    = df_dict[stock].copy()
    mdf = df[['PX_OPEN','RSI_14D','PX_TREND','PX_VTREND']]
    df['TWITTER_PUBLICATION_COUNT'].replace(to_replace = 0, value = 1, inplace=True) # Replace 0 by 1

    ss  = df['TWITTER_POS_SENTIMENT_COUNT'] / df['TWITTER_PUBLICATION_COUNT']
    mdf.loc[:,'TWEETPR'] = ss

    ss  = df['TWITTER_NEG_SENTIMENT_COUNT'] / df['TWITTER_PUBLICATION_COUNT']
    mdf.loc[:,'TWEETNR'] = ss

    df['NEWS_PUBLICATION_COUNT'].replace(to_replace = 0, value = 1, inplace=True)
    
    ss  = df['NEWS_POS_SENTIMENT_COUNT'] / df['NEWS_PUBLICATION_COUNT']
    mdf.loc[:,'NEWSPR'] = ss 

    ss  = df['NEWS_NEG_SENTIMENT_COUNT'] / df['NEWS_PUBLICATION_COUNT']
    mdf.loc[:,'NEWSNR'] = ss

    if np.isinf(mdf).values.sum() > 0: # Check if there is any infinite value in the dataframe
        pdb.set_trace()
        
    # mdf.columns
    dfY   = mdf['PX_OPEN'].copy()
    dfX   = mdf[['PX_OPEN','RSI_14D', 'PX_TREND','PX_VTREND', 'TWEETPR', 'TWEETNR', 'NEWSPR','NEWSNR']].copy()
    idx   = dfX.index[(mwin-1):]
    mX    = []
    cols= dfX.columns

    for i in range(len(dfX.columns)):
        ss = windowing(dfX.iloc[:,i],mwin,m_ftrs,idx)
        mX.append(ss.to_numpy())
    mX = np.transpose(np.stack(mX,axis=1),(0,2,1))

    for ahead in lahead:
        mXl  = mX[:-(ahead+1),:,:]
        mYl  = dfY.iloc[ahead+mwin:]
        avmX = np.mean(mXl,axis=1)
        avmXc= mXl-avmX[:,None,:]
        pavX = pd.DataFrame(np.mean(mXl,axis=1),columns=dfX.columns)
        pavX.set_index(idx[ahead+1:],drop=True,inplace=True)
        mxmX = np.round(np.max(avmXc,axis=(0,1)),2)
        mnmX = np.round(np.min(avmXc,axis=(0,1)),2)
        mvdd = {'mean':pavX,'min':mnmX,'max':mxmX}
        mXn  = (avmXc-mnmX[None,None,:])/(mxmX[None,None,:]-mnmX[None,None,:]+0.00001)
        mYn  = ((mYl.to_numpy()-avmX[:,0])-mnmX[0])/(mxmX[0]-mnmX[0]+0.00001)

        mXn = mXn.astype('float32')
        mYn = mYn.astype('float32')

        # Data set preparation for modeling, starting from cXn and cYn
        pmod   = int(mXn.shape[0]*tr_tst)  # 20% of data for testing

        mtrainX = mXn[:pmod,:,:]
        mtrainY = mYn[:pmod]
        mtestX = mXn[pmod:,:,:]
        mtestY = mYn[pmod:]
        xdx   = idx[:-(ahead+1)]
        ydx   = mYl[pmod:].index  # Taking out the indexes for plotting

        if np.isinf(mtrainX).any():
            print('Stock {} ahead: {} has nans.'.format(stock,ahead))
            print(mtrainX)
            pdb.set_trace()
        if np.isnan(mtrainX).any():
            print('Stock {} ahead: {} has nans.'.format(stock,ahead))
            print(mtrainX)
            pdb.set_trace()
        if np.isnan(mtestX).any():
            print('Stock {} ahead: {} has nans.'.format(stock,ahead))
            print(mtestX)
        mserial_dict[stock][ahead] = {'x':mXl,'y':mYl,'nx':mXn,'ny':mYn,'numt':pmod, \
                                      'vdd':mvdd, 'trX':mtrainX,'trY':mtrainY, \
                                      'tsX':mtestX,'tsY':mtestY,'cnms':cols,\
                                      'idtest':xdx[pmod:]}

tot_res['INP_MSERIAL'] = mserial_dict

In [38]:
print(tot_res['INP_MSERIAL']['TSLA'].keys()) # Check the times are 1, 7, 14, 30 and 90 days
print(tot_res['INP_MSERIAL']['TSLA'][7]['x'].shape) # Check the dimension of the data for each time

dict_keys([1, 7, 14, 30, 90])
(2224, 15, 8)


In [62]:
fdat = "data_processed/{:02}/m-input-output.pkl".format(win)

lpar = [mwin, mdeep, n_ftrs, tr_tst]

with open(fdat, 'wb') as file:
    # dump information to that file
    pickle.dump(path, file)
    pickle.dump(fdat, file)
    pickle.dump(lahead, file)
    pickle.dump(lpar, file)
    pickle.dump(stock_list, file)
    pickle.dump(tot_res, file)
file.close()

In [52]:
print(mserial_dict['TSLA'][7].keys())
print(mserial_dict['TSLA'][7]['cnms'])

dict_keys(['x', 'y', 'nx', 'ny', 'numt', 'vdd', 'trX', 'trY', 'tsX', 'tsY', 'cnms', 'idtest'])
Index(['PX_OPEN', 'RSI_14D', 'PX_TREND', 'PX_VTREND', 'TWEETPR', 'TWEETNR',
       'NEWSPR', 'NEWSNR'],
      dtype='object')


In [51]:
mserial_dict['TSLA'][7]['cnms']

Index(['PX_OPEN', 'RSI_14D', 'PX_TREND', 'PX_VTREND', 'TWEETPR', 'TWEETNR',
       'NEWSPR', 'NEWSNR'],
      dtype='object')

In [54]:
import session_info
session_info.show()